In [13]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()



  Activating environment at `~/CMU/16-745 Optimal Control/HEXROTOR_MODELING/Project.toml`


In [17]:
Pkg.add("LinearAlgebra")

   Resolving package versions...
    Updating `~/CMU/16-745 Optimal Control/HEXROTOR_MODELING/Project.toml`
  [37e2e46d] + LinearAlgebra
  No Changes to `~/CMU/16-745 Optimal Control/HEXROTOR_MODELING/Manifest.toml`


In [1]:
# hexrotor model

model_hexrotor= (mass= 6.725,
    J= Diagonal([0.0025, 0.002, 0.0045]), 
    gravity= [0,0,-9.81],
    L= 1.075, # total length from end to end
    kf= , # force coefficient
    km= , # torque coefficient
    dt= 0.05)


## assumptions
# center of mass is at the center of the drone
# mass is distributed evenly


ErrorException: syntax: unexpected ","

In [ ]:
## PARAMETERS
# number of states
nx=

# number of controls
nu= 

params = (
    N = N, 
    nx = nx, 
    nu = nu, 
    Xref = Xref, 
    Uref = Uref, 
    Q = Q, 
    R = R, 
    Qf = Qf, 
    model = model_hexrotor
)



In [ ]:
## UTILITY FUNCTIONS

function dcm_from_mrp(p)
    p1,p2,p3 = p
    den = (p1^2 + p2^2 + p3^2 + 1)^2
    a = (4*p1^2 + 4*p2^2 + 4*p3^2 - 4)
    [
    (-((8*p2^2+8*p3^2)/den-1)*den)   (8*p1*p2 + p3*a)     (8*p1*p3 - p2*a);
    (8*p1*p2 - p3*a) (-((8*p1^2 + 8*p3^2)/den - 1)*den)   (8*p2*p3 + p1*a);
    (8*p1*p3 + p2*a)  (8*p2*p3 - p1*a)  (-((8*p1^2 + 8*p2^2)/den - 1)*den)
    ]/den
end

function skew(ω::Vector)
    return [0    -ω[3]  ω[2];
            ω[3]  0    -ω[1];
           -ω[2]  ω[1]  0]
end

In [ ]:
# HEXROTOR Dynamicsfunction quadrotor_dynamics(model::NamedTuple,x,u)
    # quadrotor dynamics with an MRP for attitude
    # and velocity in the world frame (not body frame)
    
    r = x[1:3]     # position in world frame 
    v = x[4:6]     # position in body frame 
    p = x[7:9]     # n_p_b (MRP) attitude 
    ω = x[10:12]   # angular velocity 

    Q = dcm_from_mrp(p)

    mass=model.mass
    J = model.J
    gravity= model.gravity
    L= model.L
    kf=model.kf
    km=model.km

    w1 = u[1]
    w2 = u[2]
    w3 = u[3]
    w4 = u[4]
    w5 = u[5]
    w6 = u[6]

    F1 = max(0,kf*w1)
    F2 = max(0,kf*w2)
    F3 = max(0,kf*w3)
    F4 = max(0,kf*w4)
    F5 = max(0,kf*w5)
    F6 = max(0,kf*w6)
    F = [0., 0., F1+F2+F3+F4+F5+F6] #total rotor force in body frame

    M1 = km*w1
    M2 = km*w2
    M3 = km*w3
    M4 = km*w4
    M5 = km*w5
    M6 = km*w6

    ### edit this to account for 6 rotors
    τ = [L*(F2-F4), L*(F3-F1), (M1-M2+M3-M4)] #total rotor torque in body frame

    f = mass*gravity + Q*F # forces in world frame

    # this is xdot 
    [
        v
        f/mass
        ((1+norm(p)^2)/4) *(   I + 2*(skew(p)^2 + skew(p))/(1+norm(p)^2)   )*ω
        J\(τ - cross(ω,J*ω))
    ]
end

In [ ]:
# discretization with rk4
function discrete_dynamics(params::NamedTuple, x::Vector, u, k)
    # discrete dynamics
    # x - state 
    # u - control 
    # k - index of trajectory 
    # dt comes from params.model.dt 
    return rk4(params.model, quadrotor_dynamics, x, u, params.model.dt)
end